In [6]:
import requests
import pandas as pd

from lxml import etree
import time
from random import randint
from aiohttp import ClientSession,TCPConnector
import nest_asyncio
import glob
import asyncio
import numpy as np

# asyncio and jupyter cause trouble, this is a fix :
# https://markhneedham.com/blog/2019/05/10/jupyter-runtimeerror-this-event-loop-is-already-running/
nest_asyncio.apply()

In [19]:
ROOT_URL = 'https://www.franceculture.fr/emissions/les-chemins-de-la-philosophie?p={}'
range_url = range(1,2)

In [41]:
async def get_list_of_podscast_on_page(page_number,session):
    url =ROOT_URL.format(str(page_number))
    print(url)
    
    try:
        async with session.get(url) as r:
            content = await r.text()
            e = etree.HTML(content)
            list_dict = []
            elements = e.xpath("//div[@class='teaser-replay-button-wrapper']//button")
            list_dic  = [dict(d.attrib) for d in elements]
            df = pd.DataFrame(list_dic)
            return df
          
    except Exception as e:
        print(e)
        return None

In [42]:
async def run(range_url):
  
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    # we set the connector to 10 because we are well educated people who play by the rules :
    connector = TCPConnector(limit=10)
    async with ClientSession(connector=connector) as session:
        for url in range_url:
            task = asyncio.ensure_future(get_list_of_podscast_on_page(url, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks,return_exceptions=True )
        # you now have all response bodies in this variable
        return (responses)
    
def scrap_all_metadata(dt_lista):
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(run(range_url))
    df_list = loop.run_until_complete(future) 
    
    return df_list

In [43]:
dt_lista = scrap_all_metadata(range_url)
dt_lista

https://www.franceculture.fr/emissions/les-chemins-de-la-philosophie?p=1


[                       class data-btn-type data-network-change  \
 0   replay-button paused aod           aod                       
 1   replay-button paused aod           aod                       
 2   replay-button paused aod           aod                       
 3   replay-button paused aod           aod                       
 4   replay-button paused aod           aod                       
 5   replay-button paused aod           aod                       
 6   replay-button paused aod           aod                       
 7   replay-button paused aod           aod                       
 8   replay-button paused aod           aod                       
 9   replay-button paused aod           aod                       
 10  replay-button paused aod           aod                       
 11  replay-button paused aod           aod                       
 12  replay-button paused aod           aod                       
 13  replay-button paused aod           aod                   

In [45]:
dt_lista[0]

,class,data-btn-type,data-network-change,title,data-asset-title,data-asset-surtitle,aria-label,data-asset-source,data-asset-created-date,data-clip-type,data-duration,data-media-id,data-title-link,data-surtitle-link,data-business-reference,data-diffusion-id,data-short-id,data-expression-id,data-concept-id,data-universe-title
0,replay-button paused aod,aod,,Réécouter Profession philosophe (15/74) : Cath...,Profession philosophe (15/74) : Catherine Mala...,Les Chemins de la philosophie,Lecture du replay,https://media.radiofrance-podcast.net/podcast0...,1591344000,reecoute,3538,69ccccc0-543c-41a7-89cb-ba66eb2a87b2,/emissions/les-chemins-de-la-philosophie/profe...,/emissions/les-chemins-de-la-philosophie,12410,,fce82e48,69ccccc0-543c-41a7-89cb-ba66eb2a87b2,2fae969a-3f60-11e5-8314-005056a87c89,savoirs
1,replay-button paused aod,aod,,Réécouter Besoin de Bergson (4/4) : Métaphysiq...,Besoin de Bergson (4/4) : Métaphysique du mouv...,Les Chemins de la philosophie,Lecture du replay,https://media.radiofrance-podcast.net/podcast0...,1591257600,reecoute,3534,f679111b-e8c5-4d8f-8127-aebd6b248e1e,/emissions/les-chemins-de-la-philosophie/les-c...,/emissions/les-chemins-de-la-philosophie,12410,,9768ca13,f679111b-e8c5-4d8f-8127-aebd6b248e1e,2fae969a-3f60-11e5-8314-005056a87c89,savoirs
2,replay-button paused aod,aod,,Réécouter Besoin de Bergson (3/4) : Qu’est-ce ...,Besoin de Bergson (3/4) : Qu’est-ce que le mom...,Les Chemins de la philosophie,Lecture du replay,https://media.radiofrance-podcast.net/podcast0...,1591171200,reecoute,3529,b8b5634a-57f7-4a1e-a864-2bf36224f7f1,/emissions/les-chemins-de-la-philosophie/besoi...,/emissions/les-chemins-de-la-philosophie,12410,,ada9fafa,b8b5634a-57f7-4a1e-a864-2bf36224f7f1,2fae969a-3f60-11e5-8314-005056a87c89,savoirs
3,replay-button paused aod,aod,,Réécouter Besoin de Bergson (2/4) : Où dorment...,Besoin de Bergson (2/4) : Où dorment nos souve...,Les Chemins de la philosophie,Lecture du replay,https://media.radiofrance-podcast.net/podcast0...,1591084800,reecoute,3525,90feb9f3-5ab1-4b15-b898-86b159b122a5,/emissions/les-chemins-de-la-philosophie/besoi...,/emissions/les-chemins-de-la-philosophie,12410,,8ce5100c,90feb9f3-5ab1-4b15-b898-86b159b122a5,2fae969a-3f60-11e5-8314-005056a87c89,savoirs
4,replay-button paused aod,aod,,Réécouter Besoin de Bergson (1/4) : À l’origin...,Besoin de Bergson (1/4) : À l’origine du vital...,Les Chemins de la philosophie,Lecture du replay,https://media.radiofrance-podcast.net/podcast0...,1590998400,reecoute,3538,3e548b27-b66f-4c62-852a-e2f3c165ada2,/emissions/les-chemins-de-la-philosophie/besoi...,/emissions/les-chemins-de-la-philosophie,12410,,ba1889d5,3e548b27-b66f-4c62-852a-e2f3c165ada2,2fae969a-3f60-11e5-8314-005056a87c89,savoirs
5,replay-button paused aod,aod,,Réécouter Profession philosophe (10/74) : Clau...,Profession philosophe (10/74) : Claudine Tierc...,Les Chemins de la philosophie,Lecture du replay,https://media.radiofrance-podcast.net/podcast0...,1590739200,reecoute,3536,95d9e7ce-72b0-44f0-96e5-1246cd6fbb7c,/emissions/les-chemins-de-la-philosophie/les-c...,/emissions/les-chemins-de-la-philosophie,12410,,77a0823b,95d9e7ce-72b0-44f0-96e5-1246cd6fbb7c,2fae969a-3f60-11e5-8314-005056a87c89,savoirs
6,replay-button paused aod,aod,,"Réécouter Du théâtre ! (4/4) : ""La Fuite"" de B...","Du théâtre ! (4/4) : ""La Fuite"" de Boulgakov p...",Les Chemins de la philosophie,Lecture du replay,https://media.radiofrance-podcast.net/podcast0...,1590652800,reecoute,3546,b87f050b-a2c9-4f16-ad2b-9dd94401b8dc,/emissions/les-chemins-de-la-philosophie/du-th...,/emissions/les-chemins-de-la-philosophie,12410,,22ebe015,b87f050b-a2c9-4f16-ad2b-9dd94401b8dc,2fae969a-3f60-11e5-8314-005056a87c89,savoirs
7,replay-button paused aod,aod,,"Réécouter Du théâtre ! (3/4) : ""Bérénice"" de R...","Du théâtre ! (3/4) : ""Bérénice"" de Racine par ...",Les Chemins de la philosophie,Lecture du replay,https://media.radiofrance-podcast.net/podcast0...,1590566400,reecoute,3530,ff23baea-8fb7-4e58-84e5-1d858f6dcc02